In [2]:
import numpy as np
import torch
import torch.nn as nn
from itertools import permutations
import os
import argparse
import pandas as pd
import dataloaders.mrs as mrs
import torchvision.transforms as transforms

In [3]:
# COCO info
coco_info =  {
    "data_name": "coco",
    "data": "/bbfs/scratch/hyw_husy/COCO",
    "phase": "val",
    "num_classes": 80,
}


In [13]:
# load model
from dataloaders.default_coco import COCO2014Classification
from models.COCO.MLD.funcs import MLDecoder, mld_validate_multi
from models.COCO.ASL.funcs import ASL, asl_validate_multi

mld_info = {
        "model_name": "mld",
        "model_type": "tresnet_l",
        "model_path": "models/COCO/MLD/tresnet_l_COCO__448_90_0.pth",
        "workers": 4,
        "image_size": 448,
        "threshold": 0.5,
        "batch_size": 1,
        "print_freq": 10,
        "use_ml_decoder": 1,
        "num_of_groups": -1,
        "decoder_embedding": 768,
        "zsl": 0
    }

asl_info = {
        "model_name": "asl",
        "model_type": "tresnet_l",
        "model_path": "models/COCO/ASL/MS_COCO_TRresNet_L_448_86.6.pth",
        "workers": 4,
        "image_size": 448,
        "threshold": 0.8,
        "batch_size": 1,
        "print_freq": 10,
    }

def load_model(model_name):
    if model_name == 'mld':
        args = argparse.Namespace(**coco_info, **mld_info)
        model = MLDecoder(args)
    else: # model_name == 'ASL'
        args = argparse.Namespace(**coco_info, **asl_info)
        model = ASL(args)
        pass

    args.use_gpu = torch.cuda.is_available()

    return model, args

In [10]:
# get ouputs of source images
def test_source(model, args):
    pred = test(model, args, COCO2014Classification)
    if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)
    pred.to_csv(os.path.join(args.save_path, args.data_name.upper()+'_'+args.model_name.upper()+'_source.csv'), index=False)

# get outputs of followup images
def test_followup(model, args):
    if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)
    filename = os.path.join(args.save_path, args.data_name.upper()+'_'+args.model_name.upper()+'_followup.npy')
    chunk_size = 100
    num = 0
    pred_followup = {}
    if os.path.exists(filename):
        existed = np.load(filename,allow_pickle=True).item()
    else:
        existed = {}
    for k in range(1): 
        k = k+1
        for p in permutations(range(len(mrs.mrs)), k):
            print(p)
            if p in existed.keys():
                continue
            followup_test_dateset = COCO2014Classification(args.data, phase=args.phase, followup=args.followup, cmr=p)
            pred = test(model, args, followup_test_dateset)
            pred_followup[p] = pred
            
            num += 1
            if num == chunk_size:
                    pred_followup.update(existed)
                    np.save(filename, pred_followup)
                    existed = pred_followup
                    num = 0
                    pred_followup = {}

    if len(pred_followup) != 0:
        pred_followup.update(existed)
        np.save(filename, pred_followup)

def test(model, args, dataloader_class):    
    if args.model_name == 'mld':
        val_dataset = dataloader_class(
            args.data, phase=args.phase,
            followup=args.followup,
            transform=transforms.Compose([
                transforms.Resize((args.image_size, args.image_size)),
                transforms.ToTensor(),
            ])
        )
    elif args.model_name == 'asl':
        normalize = transforms.Normalize(mean=[0, 0, 0],
                                         std=[1, 1, 1])
        val_dataset = dataloader_class(
            args.data, phase=args.phase,
            followup=args.followup,
            transform=transforms.Compose([
                transforms.Resize((args.image_size, args.image_size)),
                transforms.ToTensor(),
                normalize,
            ])
        )
    else:
        raise NotImplementedError("model {} is not implemented".format(args.model_name))
    
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)

    if args.model_name == 'mld':
        pred, map = mld_validate_multi(val_loader, model, args)
    else: # args.model_name == 'asl'
        pred, map = asl_validate_multi(val_loader, model, args)
    print(len(pred))
    return pred

In [14]:
model_names = ['mld']
print(os.getcwd())
for model_name in model_names:
    print(f'----------------------------{model_name}----------------------------')
    model, args = load_model(model_name)
    args.save_path = 'predictions/COCO/'
    args.followup = False
    test_source(model, args)
    args.followup = True
    test_followup(model, args)

/home/husy/Documents/CMR
----------------------------mld----------------------------
creating model tresnet_l...
done
[dataset] Done!
[annotation] Done!
[json] Done!
starting actuall validation
Test: [0/40137]	Time 0.079 (0.079)	Precision 50.00 (50.00)	Recall 66.67 (66.67)
P_C 2.50 R_C 2.50 F_C 2.50 P_O 50.00 R_O 66.67 F_O 57.14
Test: [10/40137]	Time 0.007 (0.014)	Precision 30.77 (48.86)	Recall 80.00 (90.54)
P_C 22.95 R_C 30.52 F_C 24.81 P_O 49.56 R_O 91.80 F_O 64.37
Test: [20/40137]	Time 0.008 (0.011)	Precision 75.00 (52.20)	Recall 100.00 (94.68)
P_C 31.41 R_C 46.11 F_C 35.04 P_O 51.36 R_O 94.96 F_O 66.67
Test: [30/40137]	Time 0.008 (0.010)	Precision 66.67 (53.39)	Recall 100.00 (95.56)
P_C 36.17 R_C 52.04 F_C 40.52 P_O 51.30 R_O 95.18 F_O 66.67
Test: [40/40137]	Time 0.008 (0.010)	Precision 42.86 (53.55)	Recall 100.00 (95.55)
P_C 36.15 R_C 57.34 F_C 41.89 P_O 50.79 R_O 95.05 F_O 66.21
Test: [50/40137]	Time 0.008 (0.009)	Precision 60.00 (55.14)	Recall 100.00 (96.42)
P_C 38.59 R_C 59.94 

---------------------------------------


In [ ]:
filename = 'predictions/VOC/VOC_MCAR_followup.npy'
existed = np.load(filename,allow_pickle=True).item()
print(len(existed))
#print(existed.keys())

In [ ]:
import os

# 指定文件夹路径
folder_path = 'data/VOC/VOCdevkit/VOC2007/ImageSets/Main/'

# 遍历文件夹下的所有文件
for filename in os.listdir(folder_path):
    # 构建原文件的完整路径
    old_filepath = os.path.join(folder_path, filename)
    
    # 检查是否是文件
    if os.path.isfile(old_filepath):
        # 构建新文件名，向原文件名后面添加 "2"
        if 'pottedplant' in filename:

            new_filename = filename.replace('pottedplant', 'potted plant')
        
            # 构建新文件的完整路径
            new_filepath = os.path.join(folder_path, new_filename)
            
            # 重命名文件
            os.rename(old_filepath, new_filepath)

In [ ]:
import json

path = 'data/COCO/'
annotation_path = path + 'annotations/image_info_test2014.json'

# 加载原始COCO注释文件
with open(annotation_path, 'r') as f:
    data = json.load(f)

# 修改所有图片的扩展名从.jpg到.png
for img in data['images']:
    img['file_name'] = img['file_name'].replace('.jpg', '.png')

# 保存修改后的注释文件
modified_annotation_path = path + 'annotations/image_info_test2014_followup.json'  # 修改后的文件路径
with open(modified_annotation_path, 'w') as f:
    json.dump(data, f)

print("Modified annotation file has been saved.")

Modified annotation file has been saved.
